In [1]:
import pandas as pd
import tensorflow as tf
import numpy as np
import tensorflow.keras.backend as K
from rdkit import Chem
from tensorflow.keras.callbacks import EarlyStopping
from megnet.data.molecule import MolecularGraph
from megnet.models import MEGNetModel
from megnet.data.graph import GaussianDistance
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

C:\Users\quche\anaconda3\envs\ml\lib\site-packages\pymatgen\core\__init__.py:49: UserWarning: Error loading .pmgrc.yaml: [Errno 2] No such file or directory: 'C:\\Users\\quche\\.pmgrc.yaml'. You may need to reconfigure your yaml file.
  warnings.warn(f"Error loading .pmgrc.yaml: {ex}. You may need to reconfigure your yaml file.")


## Load data, scale y, and split the data into training, validation, and testing

In [13]:
x = [mol for mol in Chem.SDMolSupplier("data/bp_new3.sdf", removeHs=True)]
data = pd.read_csv('data/bp_new3.csv', delim_whitespace=True)
y = data['bp'].to_numpy().reshape((-1,1))
nid = data['trcid'].to_numpy(dtype=int).reshape((-1,1))

scaler = StandardScaler().fit(y)
y = scaler.transform(y)
print("Mean: ", scaler.mean_)
print("Variance: ", scaler.var_)

X_train, X_tp, y_train, y_tp, id_train, id_tp = train_test_split(x, y, nid, test_size=0.20)
X_val, X_test, y_val, y_test, id_val, id_test = train_test_split(X_tp, y_tp, id_tp, test_size=0.5)
n_train = len(X_train)
n_val   = len(X_val)
n_test  = len(X_test)
print(n_train, n_val, n_test)

Mean:  [435.13952468]
Variance:  [5406.49465949]
3080 385 385


## Select features for molecular graph; create the MEGNet model, and train

In [14]:
graph = MolecularGraph(atom_features=['element', 'hybridization', 'formal_charge'],
                       bond_features=['bond_type', 'same_ring', 'graph_distance'],
                       known_elements=['C', 'H', 'O', 'N', 'F', 'Si', 'S', 'Cl', 'Br', 'P', 'I'],
                       cutoff = 5)

model = MEGNetModel(nfeat_edge=7, nfeat_global=3, nfeat_node=18, nblocks=3, learning_rate=2e-4,
                    n1=64, n2=32, n3=16, npass=3, ntarget=1, act='relu', final_act=None,
                    graph_converter=graph, loss="mae")
es_callback = EarlyStopping(monitor='val_loss', patience=250, restore_best_weights=True)

# Train model
model.train(X_train, y_train, X_val, y_val,
            epochs=2000, batch_size=32, verbose=2, callbacks=[es_callback])

Epoch 1/2000
97/97 - 8s - loss: 0.6102 - val_loss: 0.4448 - 8s/epoch - 84ms/step
Epoch 2/2000
97/97 - 2s - loss: 0.3349 - val_loss: 0.3200 - 2s/epoch - 24ms/step
Epoch 3/2000
97/97 - 2s - loss: 0.2752 - val_loss: 0.3032 - 2s/epoch - 25ms/step
Epoch 4/2000
97/97 - 2s - loss: 0.2692 - val_loss: 0.3243 - 2s/epoch - 24ms/step
Epoch 5/2000
97/97 - 2s - loss: 0.2578 - val_loss: 0.3151 - 2s/epoch - 23ms/step
Epoch 6/2000
97/97 - 2s - loss: 0.2381 - val_loss: 0.2670 - 2s/epoch - 24ms/step
Epoch 7/2000
97/97 - 2s - loss: 0.2364 - val_loss: 0.2652 - 2s/epoch - 23ms/step
Epoch 8/2000
97/97 - 2s - loss: 0.2279 - val_loss: 0.2462 - 2s/epoch - 23ms/step
Epoch 9/2000
97/97 - 2s - loss: 0.2148 - val_loss: 0.2543 - 2s/epoch - 23ms/step
Epoch 10/2000
97/97 - 2s - loss: 0.2067 - val_loss: 0.3092 - 2s/epoch - 24ms/step
Epoch 11/2000
97/97 - 2s - loss: 0.2174 - val_loss: 0.2327 - 2s/epoch - 24ms/step
Epoch 12/2000
97/97 - 2s - loss: 0.1955 - val_loss: 0.2438 - 2s/epoch - 23ms/step
Epoch 13/2000
97/97 - 2s 

## Compute MAE and RMSE of the predictions

In [15]:
y_pred = [model.predict_structure(x) for x in X_train]
y_pred = np.array(y_pred, dtype=float).reshape(-1,1)
y_pred = scaler.inverse_transform(y_pred)
y_true = scaler.inverse_transform(y_train)
mae  = np.mean(np.abs(y_true-y_pred))
rmse = np.sqrt(np.mean(np.square(y_pred-y_true)))
print("Training MAE: ", mae)
print("Training RMSE: ", rmse)
np.savetxt("result/train.csv", np.concatenate((id_train, y_train, y_pred), axis=1), fmt='%8i %8.1f %8.1f', delimiter="   ")

y_pred = [model.predict_structure(x) for x in X_val]
y_pred = np.array(y_pred, dtype=float).reshape(-1,1)
y_pred = scaler.inverse_transform(y_pred)
y_true  = scaler.inverse_transform(y_val)
mae  = np.mean(np.abs(y_true-y_pred))
rmse = np.sqrt(np.mean(np.square(y_pred-y_true)))
print("Validation MAE: ", mae)
print("Validation RMSE: ", rmse)
np.savetxt("result/valid.csv", np.concatenate((id_val, y_val, y_pred), axis=1), fmt='%8i %8.1f %8.1f', delimiter="   ")

y_pred = [model.predict_structure(x) for x in X_test]
y_pred = np.array(y_pred, dtype=float).reshape(-1,1)
y_pred = scaler.inverse_transform(y_pred)
y_true = scaler.inverse_transform(y_test)
mae  = np.mean(np.abs(y_true-y_pred))
rmse = np.sqrt(np.mean(np.square(y_pred-y_true)))
print("Testing MAE: ", mae)
print("Testing RMSE: ", rmse)
np.savetxt("result/test.csv", np.concatenate((id_test, y_test, y_pred), axis=1), fmt='%8i %8.1f %8.1f', delimiter="   ")

Training MAE:  2.3305424112782283
Training RMSE:  3.705626601681215
Validation MAE:  6.268253582085762
Validation RMSE:  12.070565083681855
Testing MAE:  5.42643917626076
Testing RMSE:  9.249815966511644


## Save the weights of the trained model

In [17]:
model.save_weights('./checkpoints/bp_model_weights')